<a href="https://colab.research.google.com/github/Zubair2019/My-Projects/blob/main/BertBasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-tuning a pretrained Bert Model fro Classification using Pytorch

Install Required Packages

In [ ]:
!pip install datasets
!pip install transformers

Before finetuning a model we need the dataset with which to finetune the model. For this tutorial we will use the imdb dataset to classify a movie reviews as positive and negative. The raw_datasets object is a dictionary with three keys: "train", "test" and "unsupervised" (which correspond to the three splits of that dataset). We will use the "train" split for training and the "test" split for validation.

In [3]:
from datasets import load_dataset
raw_dataset = load_dataset('imdb')
print(raw_dataset)

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


Now we will use the AutoTokenizer from the transformers library to preprocess our data.

In [4]:
from transformers import AutoTokenizer
tokenizer  = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

We will use the map method for pre-processing all the splits of data

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

Lets keep a small dataset for finetuning. We will use this dataset for finetuning in this tutorial

In [6]:
small_train_dataset = tokenized_datasets['train'].shuffle(seed = 42).select(range(1000))
small_test_dataset = tokenized_datasets['test'].shuffle(seed = 42).select(range(1000))
full_train_dataset = tokenized_datasets['train']
full_test_datasets = tokenized_datasets['test']

In [26]:
import pandas as pd
print(small_train_dataset['label'][0:5],'\n')
df = pd.DataFrame(small_train_dataset)

[0, 0, 1, 0, 1] 



In [28]:
df.head(1)

,attention_mask,input_ids,label,text,token_type_ids
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1188, 1110, 1240, 4701, 153, 16383, 7858...",0,This is your typical Priyadarshan movie--a bun...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Since our downstream task is a cassification task while the Bert has been pre-trained for text Generation task, we will use the "BertForClassification" model for our down stream task. This means we are removing the Pre-trained Head of Bert and replacing it with the Classification Head. This apparently means some of the weights will be randomly initialized.

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased',num_labels = 2)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
from transformers import TrainingArguments
training_args = TrainingArguments('test_trainer', evaluation_strategy='epoch')

In [12]:
from transformers import Trainer
trainer = Trainer(model = model, args = training_args, train_dataset = small_train_dataset, eval_dataset = small_test_dataset, compute_metrics = compute_metrics)

In [10]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [14]:
trainer.train()
trainer.evaluate()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.753134,0.671000
2,No log,0.392542,0.862000
3,No log,0.526615,0.873000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'epoch': 3.0,
 'eval_accuracy': 0.873,
 'eval_loss': 0.5266148447990417,
 'eval_runtime': 70.4323,
 'eval_samples_per_second': 14.198,
 'eval_steps_per_second': 1.775}